In [1]:
import re
import pandas as pd
from datetime import datetime
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Vigers/CF/Christine Chan/Prepost Triakfta/'

In [45]:
# CHCO data
chco_dates = pd.read_csv(wd + 'Data_Cleaned/chco_registry.csv')
# Fix selected dates
chco_dates.loc[chco_dates.MRN == 1053930,'Start'] = "12/1/2019"
chco_dates.loc[chco_dates.MRN == 1695512,'Start'] = "12/12/2019"
chco_dates.loc[chco_dates.MRN == 815274,'Start'] = "12/13/2019"
chco_dates.loc[chco_dates.MRN == 1078408,'Start'] = "3/9/2020"
chco_dates.loc[chco_dates.MRN == 899717,'Start'] = "2/14/2020"
# CFRD only
chco_dates = chco_dates.loc[chco_dates['CFRD yes=1'] == 1]
# Remove those without Trikafta date
chco_dates.dropna(subset = ['Start'],inplace = True)
# Add glycemic data
chco_a1c = pd.read_csv(wd + 'Data_Cleaned/chco_glycemic.csv')
chco = pd.merge(chco_dates,chco_a1c,on = ['MRN'],how = 'left')
# Add PFTs
chco_pfts = pd.read_csv(wd + 'Data_Cleaned/chco_bmi_pft.csv')
idx = chco_pfts['MRN'].isin(chco_dates['MRN']) 
chco_pfts = chco_pfts[idx]
chco = pd.merge(chco,chco_pfts,on = ['MRN','Date'],how = 'outer')
# Add CGM
chco_cgm = pd.read_csv(wd + 'Data_Cleaned/chco_cgm.csv')
chco_cgm['CFF ID'] = [int(re.sub('_.*','',s)) for s in chco_cgm['subject_id']]
chco_cgm['Date'] = [re.sub(' .*','',s) for s in chco_cgm['date_cgm_placement']]
chco = pd.merge(chco,chco_cgm,on = ['CFF ID','Date'],how = 'outer')
# Sort
chco.sort_values(by = ['MRN','Date'],inplace = True)
# Fill missing 
fill = ['DOB','CFF ID','Start','Sex','Race','Hispanic/Latinx','Genotypes1','Genotypes2','Pancreatic Status','CFRD Diagnosis Date']
chco[fill] = chco.groupby('MRN')[fill].ffill()
chco[fill] = chco.groupby('MRN')[fill].bfill()
# Write
chco.to_csv(wd + 'Data_Cleaned/chco_final.csv',index = False)


In [3]:
# Montana data
adult_dem = pd.read_csv(wd + 'Data_Cleaned/montana_demo_adults.csv')
peds_dem = pd.read_csv(wd + 'Data_Cleaned/montana_demo_kids.csv')
montana = pd.concat([adult_dem,peds_dem])
# Add CGM
montana_cgm = pd.read_csv(wd + 'Data_Cleaned/montana_cgm.csv')
montana_cgm['CFF ID'] = [int(re.sub('_.*','',s)) for s in montana_cgm['subject_id']]
montana_cgm['Date'] = [re.sub(' .*','',s) for s in montana_cgm['date_cgm_placement']]
montana = pd.merge(montana,montana_cgm,on = ['CFF ID'],how = 'outer')
# Add A1c and OGTTs
montana_a1c = pd.read_csv(wd + 'Data_Cleaned/montana_a1c_ogtt.csv')
montana_a1c.dropna(subset = ['CFRD Status'],inplace = True)
montana = pd.merge(montana,montana_a1c,on = ['CFF ID','Date'],how = 'outer')
# Add PFTs
montana_pfts_adult = pd.read_csv(wd + 'Data_Cleaned/montana_bmi_pft_adults.csv')
montana_pfts_peds = pd.read_csv(wd + 'Data_Cleaned/montana_bmi_pft_kids.csv')
montana_pfts = pd.concat([montana_pfts_adult,montana_pfts_peds])
montana = pd.merge(montana,montana_pfts,on = ['CFF ID','Date'],how = 'outer')
# G tube
montana_gtube = pd.read_csv(wd + 'Data_Cleaned/montana_demo_adults.csv')
# Sort
montana.sort_values(by = ['CFF ID','Date'],inplace = True)
# Fill missing 
fill = ['DOB','Start','Gender','Race','Mixed Race components','Is Patient of Hispanic Origin']
montana[fill] = montana.groupby('CFF ID')[fill].ffill()
montana[fill] = montana.groupby('CFF ID')[fill].bfill()
# Write
montana.to_csv(wd + 'Data_Cleaned/montana_final.csv',index = False)

In [23]:
# Washington data
uw_dates = pd.read_csv(wd + 'Data_Cleaned/uw_trikafta_dates.csv')
uw_dates.drop_duplicates(subset='CFF ID',inplace = True)
# Glycemic data
uw_a1c = pd.read_csv(wd + 'Data_Cleaned/uw_a1c_ogtt.csv')
uw = pd.merge(uw_dates,uw_a1c,on = ['CFF ID'],how = 'outer')
# BMI
uw_bmi = pd.read_csv(wd + 'Data_Cleaned/uw_bmi.csv')
uw = pd.merge(uw,uw_bmi,on = ['CFF ID','Date'],how = 'outer')
# PFTs
uw_pft = pd.read_csv(wd + 'Data_Cleaned/uw_pft.csv')
uw = pd.merge(uw,uw_pft,on = ['CFF ID','Date','Patients height'],how = 'outer')
# Demographics
uw_dem = pd.read_csv(wd + 'Data_Cleaned/uw_demo.csv')
uw = pd.merge(uw,uw_dem,on = ['CFF ID'],how = 'outer')
# Write
uw.to_csv(wd + 'Data_Cleaned/uw_final.csv',index = False)